In [2]:
import pandas as pd

# Cows

In [3]:
df_cows = pd.read_parquet('../data/input/cows.parquet')

In [ ]:
df_cows.info()

In [ ]:
df_cows.head()

In [ ]:
# Group by 'name', count occurrences, and sort descending
df_cows.groupby('name').size().sort_values(ascending=False)

In [ ]:
df_cows[df_cows['name'].isin(['Jennifer #7', 'Nicole #4', 'Mary #8'])].sort_values('name')

In [ ]:
df_cows.groupby('birthdate').size().sort_values(ascending=False)

## Conclusions
- 128 cows
- No Nulls for any column
- Names are not unique
- Birthdates are disperse between 2018 and 2022

# Sensors

In [4]:
df_sensors = pd.read_parquet('../data/input/sensors.parquet')

In [ ]:
df_sensors.info()

In [ ]:
df_sensors.head()

In [ ]:
df_sensors.groupby('unit').size().sort_values(ascending=False)

## Conclusions
- 200 different sensors
- No descriptions, just id and unit
- Just two different units: L and kg
- Negative Liters and Kilograms do not make sense. 

# Measurements

In [5]:
df_measurements = pd.read_parquet('../data/input/measurements.parquet')

In [6]:
df_measurements.info()

<class 'pandas.core.frame.DataFrame'>
Index: 563627 entries, 226539 to 258223
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sensor_id  563627 non-null  object 
 1   cow_id     563627 non-null  object 
 2   timestamp  563627 non-null  float64
 3   value      560234 non-null  float64
dtypes: float64(2), object(2)
memory usage: 21.5+ MB


In [7]:
df_measurements.head()

,sensor_id,cow_id,timestamp,value
226539,77a60975-bbd5-481f-b509-98480b70c9fd,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594462e+09,549.95
226540,0c0bc9b0-6572-4ee8-bfa0-908ba19b5c44,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594476e+09,4.71
226541,0feab3cf-b258-4718-a985-2acad03b9aca,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594490e+09,549.87
226542,6caa6864-80e0-4c13-92a0-8d84b9c0d324,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594505e+09,4.65
226543,03b9a22b-15f3-4d89-b9a4-8137a39077e8,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594519e+09,4.71


In [ ]:
df_measurements.groupby('sensor_id').size().sort_values(ascending=False)

In [ ]:
df_measurements.groupby('cow_id').size().sort_values(ascending=False)

In [ ]:
df_measurements[df_measurements['timestamp'].isna()]

In [ ]:
df_measurements[df_measurements['value'].isna()]

In [ ]:
df_measurements[df_measurements['value'] < 0]

In [ ]:
first_birth = pd.Timestamp("2018-07-25")
df_measurements[(pd.to_datetime(df_measurements["timestamp"], unit="s") < first_birth)]

In [ ]:
df_measurements["timestamp"] = pd.to_datetime(df_measurements["timestamp"], unit="s")
joined_cows = df_measurements.merge(df_cows, left_on="cow_id", right_on="id", suffixes=("_meas", "_cow"))
filtered = joined_cows[joined_cows["birthdate"] <= joined_cows["timestamp"]]
filtered

In [8]:
# Join measurements with sensors by sensor_id and filter by unit range
joined_sensors = df_measurements.merge(df_sensors, left_on="sensor_id", right_on="id", suffixes=("_meas", "_sens"))

range_kg = (200, 900)
range_l = (0, 50)

kg_notvalid = joined_sensors[(joined_sensors["unit"] == "kg") & 
                          ((joined_sensors["value"] < range_kg[0]) | (joined_sensors["value"] > range_kg[1]))]
l_notvalid = joined_sensors[(joined_sensors["unit"] == "L") & 
                         ((joined_sensors["value"] < range_l[0]) | (joined_sensors["value"] > range_l[1]))]
pd.concat([kg_notvalid, l_notvalid])

,sensor_id,cow_id,timestamp,value,id,unit
22,3459d3dd-b662-40eb-931e-931701cbeef7,3321b86f-3afb-4971-a773-3ae7c62bafec,1.594778e+09,0.0,3459d3dd-b662-40eb-931e-931701cbeef7,kg
395,74932a2e-fb29-4d57-bdff-1acfafb71398,dfce91b2-2f21-4d68-b613-3c0dc36160de,1.596737e+09,0.0,74932a2e-fb29-4d57-bdff-1acfafb71398,kg
424,3459d3dd-b662-40eb-931e-931701cbeef7,dfce91b2-2f21-4d68-b613-3c0dc36160de,1.596823e+09,-1.0,3459d3dd-b662-40eb-931e-931701cbeef7,kg
441,91440d7d-8a85-4f13-92f1-0225add2e538,68923fdd-d829-4985-8851-546613c6e078,1.596866e+09,0.0,91440d7d-8a85-4f13-92f1-0225add2e538,kg
593,91440d7d-8a85-4f13-92f1-0225add2e538,edb28bf0-bf09-4469-b9b3-7d2a1dd45ff1,1.597298e+09,0.0,91440d7d-8a85-4f13-92f1-0225add2e538,kg
...,...,...,...,...,...,...
559518,98962937-0a24-41db-bdab-c7d2d73e56ef,9c6cd124-db49-45cf-8973-b47419c74518,1.720231e+09,-1.0,98962937-0a24-41db-bdab-c7d2d73e56ef,L
559651,98962937-0a24-41db-bdab-c7d2d73e56ef,47e64bdf-1ae8-4e44-98b5-3250dd2c787c,1.720246e+09,-1.0,98962937-0a24-41db-bdab-c7d2d73e56ef,L
561729,98962937-0a24-41db-bdab-c7d2d73e56ef,844fc847-a0aa-4473-a42d-d4c13cd1eba9,1.720476e+09,-1.0,98962937-0a24-41db-bdab-c7d2d73e56ef,L
563344,98962937-0a24-41db-bdab-c7d2d73e56ef,dd57c558-d620-4d28-83a7-6370f4a61322,1.720663e+09,-1.0,98962937-0a24-41db-bdab-c7d2d73e56ef,L


In [9]:
l_notvalid.groupby('value').size().sort_values(ascending=False)

value
-1.0    678
dtype: int64

In [10]:
kg_notvalid.groupby('value').size().sort_values(ascending=False)

value
 0.0    5627
-1.0    2747
dtype: int64

In [ ]:

pd.set_option('display.max_rows', None)
joined_sensors[joined_sensors['sensor_id'] == '3459d3dd-b662-40eb-931e-931701cbeef7'].sort_values('timestamp', ascending=True)

In [ ]:
joined_sensors[joined_sensors['sensor_id'] == '3459d3dd-b662-40eb-931e-931701cbeef7'].groupby('cow_id').size().sort_values(ascending=False)

In [ ]:
joined_sensors[joined_sensors['cow_id'] == '7255f6ed-8b6e-4fa5-b6bf-676663ba7e48'].groupby(['sensor_id', 'unit']).size().sort_values(ascending=False)

In [ ]:
joined_sensors[(joined_sensors['unit'] == 'kg') & (joined_sensors['value'].notna()) & (~joined_sensors['value'].isin([-1,0]))]['value'].min()

np.float64(452.36)

In [ ]:
joined_sensors[(joined_sensors['unit'] == 'kg') & (joined_sensors['value'].notna()) & (~joined_sensors['value'].isin([-1,0]))]['value'].max()

np.float64(570.18)

In [24]:
joined_sensors[(joined_sensors['unit'] == 'L') & (joined_sensors['value'].notna()) & (~joined_sensors['value'].isin([-1,0]))]['value'].min()

np.float64(4.5)

In [25]:
joined_sensors[(joined_sensors['unit'] == 'L') & (joined_sensors['value'].notna()) & (~joined_sensors['value'].isin([-1,0]))]['value'].max()

np.float64(5.0)

## Conclusions
- 500K records
- Not nulls at cow_id, sensor_id or timestamp
- Same cow uses diverse sensors, and different sensors measure diverse cows
- 3K nulls (NaN) for value
- 3K negatives for value, which don't make sense and could be checked
- No measurements timestamps previous to birthdate for its respective cow, but this could potentially be a check
- For unit=L: 678 negative values (-1) 
- For unit=kg: 5627 zeros, 2747 negative values (-1)
- Do these nulls, zeros or negatives provide any info? Looking at the time series they look like broken sensors
- Cows weight ranges between 450 to 570
- Cows milk extraction ranges between 4.5 and 5